In [3]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import re
import requests
import pickle

In [3]:
BASE = 'https://www.mass.gov'
start = 'https://www.mass.gov/guides/list-of-court-forms-by-topic'
connection = requests.get(start)
html = connection.text
soup = bs4.BeautifulSoup(html,"lxml")
# print(soup)
connection.close()
div = soup.findAll("div", class_ = "ma__rich-text")

In [4]:
hreflist = []
for category in div:
    type_of_form = category.ul
    if type_of_form!=None:
        for a in type_of_form.findAll("a"):
            hreflist.append(a['href'])
            
                        
print(hreflist)

['/lists/probate-and-family-court-forms-for-wills-estates-and-trusts#accounting-forms-', '/lists/probate-and-family-court-forms-for-actions-involving-unmarried-parents', '/lists/probate-and-family-court-forms-for-adoption-and-name-change', '/lists/attorney-court-forms', '/seeking-admission-to-the-bar-in-massachusetts', '/lists/juvenile-court-forms#child-requiring-assistance-(cra)-', '/lists/probate-and-family-court-forms-for-child-support-guidelines', '/lists/probate-and-family-court-forms-for-modifications', '/lists/court-forms-for-civil-matters', '/lists/probate-family-court-forms-for-guardianship-and-conservatorship', '/lists/court-forms-for-consumer-debt-collection-civil-rules-81-and-551', '/lists/probate-and-family-court-forms-for-contempt-actions', '/lists/juvenile-court-forms#court-investigators-&-guardian-ad-litem-', '/lists/court-forms-for-criminal-matters', '/lists/court-forms-for-criminal-records', '/lists/court-forms-for-consumer-debt-collection-civil-rules-81-and-551', '/l

In [5]:
formcategory = BASE + hreflist[0]
# print(formcategory)
connection = requests.get(formcategory)
html = connection.text
soup = bs4.BeautifulSoup(html,"lxml")
print(formcategory)
connection.close()


https://www.mass.gov/lists/probate-and-family-court-forms-for-wills-estates-and-trusts#accounting-forms-


In [2]:
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import date
import os
from os import walk
import os.path
from os import path
import uuid

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()

    import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup

In [7]:
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)
    

In [8]:
# infile = open("C_pdfs.pickle",'rb')
# unpickled_c = pickle.load(infile)
# infile.close()
# print(len(unpickled_c))

# infile = open("PDFS/A/A_pdfs.pickle",'rb')
# unpickled_a = pickle.load(infile)
# infile.close()
# print(len(unpickled_a))
# unpickled_a.update(unpickled_c)
# print(len(unpickled_a))

errors = {}
pdfs = {}
redirections = []

In [13]:
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","status"])
form_dest = "PDFS/D/"
today = date.today().strftime("%Y-%m-%d")
#remove the parenthesis
pattern = r"\([^()]*\)"
source = "https://www.mass.gov/guides/list-of-court-forms-by-topic"
jur = "MA"
status = 0
for href in range(len(hreflist)):
    time.sleep(2)
    formcategory = BASE + hreflist[href]
    connection = requests.get(formcategory)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    print(formcategory)
    connection.close()
    try:
        groups = soup.find("div",class_ = "ma__stacked-row stacked-row-column")
        section = groups.find_all("h2")
        links = soup.find_all("div", class_ = "ma__form-downloads__links")
    except:
        continue
    
    for group in range(len(section)-1): #-1 because last is contact
            forms = links[group].find_all(class_ = "ma__download-link__file-link")
            redirects = links[group].find_all(class_ = "js-clickable-link")
            curr_group = section[group].text
            print(curr_group)
            
            # redirects result into an error download
            if redirects!=None:
                for i in redirects:
                    if "https" not in i['href']:
                        inner_url = BASE + i['href']
                        time.sleep(2)
                        inner_soup = connect_to_page(inner_url)
                        inner_div = inner_soup.findAll("a", href= True)
                        for i in inner_div:
                            if "courtforms" in i['href']:
                                url = i['href']
                                title = re.sub(pattern,"",i.text)
                                filename = url.split("//")[1].split("/")[-1] +".pdf"
                                
                                if filename not in pdfs:
                                    print(title)
                                    print(filename)
                                    pdfs[filename] = title
                                    fileid = hashme(url)
                                    status = download_pdf(url,form_dest,fileid +".pdf")
                                    
                                    if status=='error':
                                        print(status, url)
                                        status = 1
                                        errors[filename] = title
                                    else: status = 0
                                    files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,curr_group,url,filename,today,status]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","status"]))
                                status = 0
                    else:
                        redirections.append(i['href'])


            for f in forms:
                url = f['href']
                filename = url.split("//")[1].split("/")[-2] +".pdf"
                
                if filename not in pdfs:
                    title = re.sub(pattern,"",f.text.split(",")[2].split("\n")[1].lstrip())
                    pdfs[filename] = title
                    fileid = hashme(url)
                    #print(curr_group)
                    print(title)
                    print(filename)
                    status = download_pdf(url,form_dest,fileid+".pdf")
                    
                    if status=='error':
                        print(status, url)
                        status = 1
                        errors[filename] = title
                    else: status = 0
                    files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,curr_group,url,filename,today,status]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","status"]))
                status = 0
if len(files_df) > 0:
    files_df.to_csv("All.csv",index=False, encoding="utf-8")

# with open('C_errors.pickle', 'wb') as error_file:
#     pickle.dump(errors, error_file)
# with open('C_pdfs.pickle', 'wb') as pdfs_file:
#     pickle.dump(pdfs, pdfs_file)
# with open('C_redirections.pickle', 'wb') as inner_sites_file:
#     pickle.dump(redirections, inner_sites_file)

https://www.mass.gov/lists/probate-and-family-court-forms-for-wills-estates-and-trusts#accounting-forms-
Accounting forms 
  
Petition to Render Inventory, Account, Distribution 
PFC0042.pdf
error https://courtforms.jud.state.ma.us/publicforms/PFC0042
Petition for Allowance of Account 
PFC0037.pdf
error https://courtforms.jud.state.ma.us/publicforms/PFC0037
Decree and Order of Allowance of Account 
PFC0064.pdf
error https://courtforms.jud.state.ma.us/publicforms/PFC0064
Order to Render 
order-to-render-mpc-754.pdf
Account 
account-mpc-853.pdf
Account Without Schedules 
account-without-schedules-mpc-853a.pdf
Trust Account 
trust-account-mpc-859.pdf
Inventory 
inventory-mpc-854.pdf
Combined Inventory Without Schedules 
combined-inventory-without-schedules-mpc-854a.pdf
Bond forms
  
Bond 
PFC0008.pdf
error https://courtforms.jud.state.ma.us/publicforms/PFC0008
Petition to Modify Bond 
petition-to-modify-bond-mpc-295.pdf
Decree and Order to Modify Bond 
decree-and-order-to-modify-bond-mpc-

In [9]:
import urllib.request
def download(download_url, filename):
    response = urllib.request.urlopen(download_url)
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import webbrowser
import io
import os
from pathlib import Path
import glob
import pandas as pd
import time

In [4]:


# a bit of cleaning
dest_sample = 'PDFS/'
df = pd.read_csv('All.csv')
df_errors = df.loc[df['status']==1]
url_list = list(df_errors['url'])
ids_list = list(df_errors['id'])
index_list = df_errors.index

print(len(df_errors))


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


hash_list = []
for url in range(len(url_list)):
    time.sleep(2)
    if 'www' not in url_list[url]:
        print(url_list[url], url)
        driver.get(url_list[url])
        
        try:
            button = driver.find_element_by_css_selector('widget.buttonfieldwidget.xfaButton > input')
            print(button)
            button.click()
            df.at[index_list[url],'status'] = 0
            hash_list.append(hashme(url_list[url]))
        except:
            print('######################')
            print('ERROR')
            print('######################')
df.to_csv("All_cleaned2.csv",index=False, encoding="utf-8")



# for url in range(len(url_list)):
#     if 'download'==url_list[url].split('/')[-1]:
#         print(url_list[url], index_list[url])
#         status = download_pdf_unmasked(url_list[url],dest_sample,ids_list[url]+'.pdf')
#         if status == 'error':
#             print('#######################')
#             print('ERROR')
#             print('#######################')
#         else:
#             df.at[index_list[url],'status'] = 0

# df.to_csv("All_cleaned.csv",index=False, encoding="utf-8")




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


77
https://courtforms.jud.state.ma.us/publicforms/PFC0042 0


C:\Users\Owner\AppData\Local\Temp\ipykernel_27172\2090484818.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button = driver.find_element_by_css_selector('widget.buttonfieldwidget.xfaButton > input')


######################
ERROR
######################
https://courtforms.jud.state.ma.us/publicforms/PFC0037 1
######################
ERROR
######################
https://courtforms.jud.state.ma.us/publicforms/PFC0064 2
######################
ERROR
######################
https://courtforms.jud.state.ma.us/publicforms/PFC0008 3
######################
ERROR
######################
https://courtforms.jud.state.ma.us/publicforms/PFC0029 4


In [13]:
pattern = r"\([^()]*\)"
folder = 'C:/Users/Owner/Downloads'
file_list = []

# paths = sorted(Path(folder).iterdir(), key=os.path.getmtime)
# print(paths)
# title = re.sub(pattern,"",i.text)
# for count, filename in enumerate(os.listdir(paths)):
#     # file_list.append(filename)
#     print(count,filename)
    # dst = f"{ids_list[file]}.pdf"
    # print(dst)
    
    # src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
    # dst =f"{folder}/{dst}"
         
    # rename() function will
    # rename all the files
    # os.rename(src, dst)

print(hash_list[30])

# os.chdir(folder)
# files = glob.glob("*.pdf")
# files.sort(key=os.path.getmtime)
# for file in range(len(files)):
#     print(file)
    # filename = os.fsdecode(files[file])
    # print(filename)
    # # file_title = filename.split('\\')[1]
    # dst = f"{ids_list[file]}.pdf"
    # src =f"{folder}/{filename}"
    # os.rename(src, dst)
# https://courtforms.jud.state.ma.us/publicforms/PFC0037 1
# https://courtforms.jud.state.ma.us/publicforms/PFC0064 2
# https://courtforms.jud.state.ma.us/publicforms/PFC0008 3
# https://courtforms.jud.state.ma.us/publicforms/PFC0029 4
# https://courtforms.jud.state.ma.us/publicforms/PFC0041 5
# https://courtforms.jud.state.ma.us/publicforms/PFC0034 6
# https://courtforms.jud.state.ma.us/publicforms/PFC0063 7
# https://courtforms.jud.state.ma.us/publicforms/PFC0015 8
# https://courtforms.jud.state.ma.us/publicforms/PFC0044 9
# https://courtforms.jud.state.ma.us/publicforms/PFC0009 10
# https://courtforms.jud.state.ma.us/publicforms/PFC0016 11
# https://courtforms.jud.state.ma.us/publicforms/PFC0012 12
# https://courtforms.jud.state.ma.us/publicforms/PFC0039 13
# https://courtforms.jud.state.ma.us/publicforms/PFC0036 14
# https://courtforms.jud.state.ma.us/publicforms/PFC0006 15
# https://courtforms.jud.state.ma.us/publicforms/PFC0067 16
# https://courtforms.jud.state.ma.us/publicforms/PFC0027 17
# https://courtforms.jud.state.ma.us/publicforms/PFC0010 18
# https://courtforms.jud.state.ma.us/publicforms/PFC0020 19
# https://courtforms.jud.state.ma.us/publicforms/PFC0025 20
# https://courtforms.jud.state.ma.us/publicforms/PFC0007 21
# https://courtforms.jud.state.ma.us/publicforms/PFC0026 22
# https://courtforms.jud.state.ma.us/publicforms/PFC0075 23

35


In [2]:
import hashlib
from selenium.webdriver.common.by import By
def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import webbrowser
import io
import os
from pathlib import Path
import glob
import pandas as pd
import time

In [30]:

dest_sample = 'PDFS/'
df = pd.read_csv('All.csv')
df_errors = df.loc[df['status']==1]
url_list = list(df_errors['url'])
ids_list = list(df_errors['id'])
index_list = df_errors.index

#print(index_list)


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

hash_list = []
for url in range(76,len(url_list)):
    time.sleep(2)
    if 'www' not in url_list[url]:
        print(url_list[url], url)
        driver.get(url_list[url])
        
        driver.find_element_by_css_selector("[aria-label='Save as PDF'").click()#this is me most common layout. However, not nearly the only one..
        #as a result some where manually downloaded
        # df.at[index_list[url],'status'] = 0

#20, 31, 33, 37, 41



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\98.0.4758.80\chromedriver.exe] found in cache


https://courtforms.jud.state.ma.us/publicforms/PFC0078 76


C:\Users\Owner\AppData\Local\Temp\ipykernel_38424\915889404.py:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("[aria-label='Save as PDF'").click()#this is me most common layout. However, not nearly the only one..


In [17]:
folder = 'C:/Users/Owner/Downloads'
df = pd.read_csv('All_cleaned2.csv')
df_errors = df.loc[df['status']==1]
url_list = list(df_errors['url'])
ids_list = list(df_errors['id'])
index_list = df_errors.index

len(index_list)

76

In [4]:

folder = 'C:/Users/Owner/Downloads'
errors_set = set(list([20, 31, 33, 37, 41])) #these were unsuccesful attempts. 2 were docs and the other 4 needed to be filled out before downloading
successful_ids = []

for id in range(len(ids_list)):
    if id not in errors_set:
        successful_ids.append(ids_list[id])
    # else:
    #     print(ids_list[id])
# for i in errors_set:
#     print(ids_list[i])

# navigate in our downloads directory and rename the files we just downloaded
# os.chdir(folder)
# files = glob.glob("*.pdf") #but only the pdfs 
# files.sort(key=os.path.getmtime) # from the first (download) to the last so it is consistent with our database
# for id in range(len(successful_ids)):
   
#     filename = os.fsdecode(files[id])
#     print(filename)
#     dst = f"{successful_ids[id]}.pdf" 
#     src =f"{folder}/{filename}"
#     os.rename(src, dst)

In [6]:
df = pd.read_csv('All.csv')
df_errors = df.loc[df['status']==1]
url_list = list(df_errors['url'])
ids_list = list(df_errors['id'])
index_list = df_errors.index

# to_change = df[df['id']==successful_ids[6]]

for id in range(len(successful_ids)):
    to_change = df[df['id']==successful_ids[id]].index
    df.at[to_change,'status'] = 0
df.to_csv("Trial.csv",index=False, encoding="utf-8")

In [4]:
retval = os.getcwd()
print(retval)
folder = 'PDFS/G'
os.chdir(folder)
files = glob.glob("*.pdf")
for file in files:
    print(file)
retval = os.getcwd()
os.chdir(retval)

c:\Users\Owner\OneDrive\Υπολογιστής\LIT
03ff8fc4e8e9168c47c61ce857f53ba1.pdf
09421b53ae01890a5bfe5d48fb41f937.pdf
09494b9c8f26fd1d3748900f525e768c.pdf
09cd22913b272e4e60d59c14d388fbda.pdf
10c5c46d0d0b1712442122fbdfa01980.pdf
121fc167faa09d7a0c33831bb7b9dcdd.pdf
1644fb48e9c2c6ebf2a35e9ed609ac1d.pdf
17eb8758f02ccff1159a523a13229a15.pdf
192751f62acf65c5c2166008c0cbd370.pdf
1ee29baf037da4c60dc04ca70f2a4a1f.pdf
232a8ad9e01a7f578b31582474608489.pdf
2943546b42ecfb4f63f89f55e5fe6256.pdf
2f6e28249ec0b137f4535412be58bde8.pdf
2fcc5fc063b795d4eb374954954da9e6.pdf
327a89994d1fedf2a88982046e83173c.pdf
3392d95879515eeef21b797726e16e1e.pdf
33d719afde61d46228c72914e3e04119.pdf
3d0f229630dcc7a2805e5027b337ee4d.pdf
42976a87d68aeb2b6b912dec0079e85b.pdf
43b02307278ebafb6492ef928903646c.pdf
471507fe406ac6bef098b0ba42dc4c17.pdf
476ee0e5d057d94c05567a6ca7118cd7.pdf
4d095ad65854ae90e0d0a8d81e89b42f.pdf
557080ac38b381078e15493bbf409e2d.pdf
5852a4de8c994bba975be406d08f8fed.pdf
5ce8061f9ae91e3a58a07cfb92174ab6.pd